# ETL III Pair 24/08

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [11]:
import pandas as pd
import requests
from datetime import datetime

In [12]:
class DatosEnergia:
    
    def __init__(self, start_year = 2011, end_year = 2022, path_to_save = "data"):
        self.start_year = start_year
        self.end_year = end_year
        self.path_to_save = path_to_save
        self.ccaa = {
            'Ceuta': 8744,
            'Melilla': 8745,
            'Andalucía': 4,
            'Aragón': 5,
            'Cantabria': 6,
            'Castilla - La Mancha': 7,
            'Castilla y León': 8,
            'Cataluña': 9,
            'País Vasco': 10,
            'Principado de Asturias': 11,
            'Comunidad de Madrid': 13,
            'Comunidad Foral de Navarra': 14,
            'Comunitat Valenciana': 15,
            'Extremadura': 16,
            'Galicia': 17,
            'Illes Balears': 8743,
            'Canarias': 8742,
            'Región de Murcia': 21,
            'La Rioja': 20
        }
        
    def info_energia_nacional(self):
        df_final = pd.DataFrame()
        for year in range(self.start_year, self.end_year + 1):
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day"
            response = requests.get(url=url)
            status_code = response.status_code
                    
            if status_code == 200:
                response_data = response.json()['included']
                len_response_data = len(response_data)
                
                for i in range(len_response_data):
                    attributes_data = response_data[i]['attributes']
                    type_energy = attributes_data['type']
                    values_data = attributes_data['values']
                    df = pd.json_normalize(values_data)
                    df["tipo_energia"] = type_energy
                    df_final = pd.concat([df_final, df], axis = 0)
            else:
                continue
        return df_final.reset_index(drop = True)
    
    def info_energia_ccaa(self):
        df_final = pd.DataFrame()
        for year in range(self.start_year, self.end_year + 1):
            for k_nombre_ccaa, v_id_ccaa in self.ccaa.items():
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day&geo_limit=ccaa&geo_ids={v_id_ccaa}"
                response = requests.get(url=url)
                status_code = response.status_code
                
                if status_code == 200:
                    response_data = response.json()['included']
                    len_response_data = len(response_data)
                    
                    for i in range(len_response_data):
                        attributes_data = response_data[i]['attributes']
                        type_energy = attributes_data['type']
                        values_data = attributes_data['values']
                        df = pd.json_normalize(values_data)
                        df["tipo_energia"] = type_energy
                        df["comunidad"] = k_nombre_ccaa
                        df["id_comunidad"] = v_id_ccaa
                        df_final = pd.concat([df_final, df], axis = 0)
                else:
                    continue
        return df_final.reset_index(drop = True)
            
    def drop_column(self, df, col):
        return df.drop(columns = [col], axis = 0, inplace = True )
        
    def round_two(self, df, columns):
        for col in columns:
            df[col] = df[col].round(2)
    
    def change_to_datetime(self, df, datetimeCol, newCol):
        df[newCol] = df[datetimeCol].str.split("T", n = 1, expand = True).get(0).astype("datetime64", errors = "ignore")
        df[newCol] = pd.to_datetime(df[newCol])
         
    def open_csv(self, path, file_name, years):
        if years:
            return pd.read_csv(f"{path}/{file_name}_{self.start_year}_{self.end_year}.csv")
        else:
            return pd.read_csv(f"{path}/{file_name}.csv")
    
    def open_pickle(self, path, file_name, years):
        if years:
            return pd.read_pickle(f"{path}/{file_name}_{self.start_year}_{self.end_year}.pkl")
        else:
            return pd.read_pickle(f"{path}/{file_name}.pkl")
           
    def save_to_csv(self, df, file_name, years):
        if years:
            df.to_csv(f"{self.path_to_save}/{file_name}_{self.start_year}_{self.end_year}.csv")
        else:
            df.to_csv(f"{self.path_to_save}/{file_name}.csv")
        
    def save_to_pickle(self, df, file_name, years):
        if years:
            df.to_pickle(f"{self.path_to_save}/{file_name}_{self.start_year}_{self.end_year}.pkl")
        else:
            df.to_pickle(f"{self.path_to_save}/{file_name}.pkl")


## Aplicamos clase

### Datos Población Comunidades

In [13]:
clase_pair3_poblacion_ccaa = DatosEnergia()
df_pair3_poblacion_ccaa = clase_pair3_poblacion_ccaa.open_csv("data","poblacion_comunidades",False)
df_pair3_poblacion_ccaa.head()

,Unnamed: 0,Comunidad,indice,Comunidades_y_Ciudades_Autónomas,Año,Total
0,0,Ceuta,8744,Ceuta,2022,83.117
1,1,Ceuta,8744,Ceuta,2021,83.517
2,2,Ceuta,8744,Ceuta,2020,84.202
3,3,Ceuta,8744,Ceuta,2019,84.777
4,4,Ceuta,8744,Ceuta,2018,85.144


In [14]:
clase_pair3_poblacion_ccaa.drop_column(df_pair3_poblacion_ccaa, "Comunidades_y_Ciudades_Autónomas")
df_pair3_poblacion_ccaa.columns

Index(['Unnamed: 0', 'Comunidad', 'indice', 'Año', 'Total'], dtype='object')

In [15]:
clase_pair3_poblacion_ccaa.save_to_csv(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", False)
clase_pair3_poblacion_ccaa.save_to_csv(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", True)
clase_pair3_poblacion_ccaa.save_to_pickle(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", False)
clase_pair3_poblacion_ccaa.save_to_pickle(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", True)

In [10]:
datos_energia = DatosEnergia(2011,2011)
datos_energia.info_energia_ccaa()

,value,percentage,datetime,tipo_energia,comunidad,id_comunidad
0,460.046,1.000000,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
1,462.983,1.000000,2011-01-02T00:00:00.000+01:00,No renovable,Ceuta,8744
2,537.072,1.000000,2011-01-03T00:00:00.000+01:00,No renovable,Ceuta,8744
3,539.194,1.000000,2011-01-04T00:00:00.000+01:00,No renovable,Ceuta,8744
4,536.034,1.000000,2011-01-05T00:00:00.000+01:00,No renovable,Ceuta,8744
...,...,...,...,...,...,...
2910,85153.431,0.544439,2011-08-01T00:00:00.000+02:00,No renovable,La Rioja,20
2911,157476.842,0.731010,2011-09-01T00:00:00.000+02:00,No renovable,La Rioja,20
2912,81134.713,0.455075,2011-10-01T00:00:00.000+02:00,No renovable,La Rioja,20
2913,70655.666,0.402930,2011-11-01T00:00:00.000+01:00,No renovable,La Rioja,20
